In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00


# Visual Transformers (ViT)


In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

# https://huggingface.co/google/vit-base-patch16-224
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits

# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: Egyptian cat


In [ ]:
# Examples of pretrained models
# https://huggingface.co/abhilash88/face-emotion-detection
# https://huggingface.co/AMfeta99/vit-base-oxford-brain-tumor_x-ray

# Whisper

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")


In [ ]:
example = "en-AU-NatashaNeural.mp3"

result = pipe(example, return_timestamps=True)
print(result["text"])

In [ ]:
from IPython.display import Javascript, display
from google.colab import output
import base64

def record(sec=10, filename="myvoice.wav"):
    display(Javascript("""
        async function record(sec){
          const sleep = time => new Promise(resolve => setTimeout(resolve, time));
          const stream = await navigator.mediaDevices.getUserMedia({audio:true});
          const recorder = new MediaRecorder(stream);
          let data = [];

          recorder.ondataavailable = event => data.push(event.data);
          alert("Start recording ...")
          recorder.start();

          await sleep(sec * 1000);
          recorder.stop();
          alert("Done .. Saving in progress ...")
          await new Promise(resolve => recorder.onstop = resolve);

          const blob = new Blob(data);
          const reader = new FileReader();
          reader.readAsDataURL(blob);

          reader.onloadend = function() {
            google.colab.kernel.invokeFunction('notebook.receive_audio', [reader.result], {});
          };
        }
        record(%d);
    """ % sec))

    # callback in Python
    def _callback(audio_data):
        audio_bytes = base64.b64decode(audio_data.split(',')[1])
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        print("Audio saved to", filename)

    output.register_callback('notebook.receive_audio', _callback)

print("Recorder ready.")


In [ ]:
record()

In [ ]:
result = pipe("myvoice.wav", return_timestamps=True)
result["text"]

# CLIP

In [ ]:
clip = pipeline(
   task="zero-shot-image-classification",
   model="openai/clip-vit-base-patch32",
   dtype=torch.bfloat16,
   device=0
)


In [ ]:
labels = ["a photo of a cat", "a photo of a dog", "a photo of a dog and a cat", "a photo of a car"]
clip("http://images.cocodataset.org/val2017/000000039769.jpg", candidate_labels=labels)

In [ ]:
clip("car.jpg", candidate_labels=labels)

In [ ]:
clip("Dog_Breeds.jpg", candidate_labels=labels)

In [ ]:
clip("dog_and_cat.jpg", candidate_labels=labels)

In [ ]:
plate_labels = ["text on the plate is: HH CT 3874",
                "text on the plate is: HH CT 3674",
                "text on the plate is: L QF 2255",
                "text on the plate is: B NE 0001",]
clip("plate.jpg", candidate_labels=plate_labels)